<a href="https://colab.research.google.com/github/MnCSSJ4x/VR-MiniProject/blob/main/VR3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
from torch import optim, cuda
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
# from torchsummary import summary
import matplotlib.pyplot as plt
%matplotlib inline

import math 
from torch.utils.data import Dataset, TensorDataset, random_split
from torchvision import transforms
from tqdm import tqdm 
import tarfile
!pip install imutils

import matplotlib.pyplot as plt
import matplotlib
import argparse 
import joblib
import cv2
import os
import time 
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25834 sha256=1cedae86ecd0ed5ec2f91d094eeea336fa82603fe5a57a5bfda609ae87676d51
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [2]:
cuda = cuda.is_available()
print(f'Train on gpu: {cuda}')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Train on gpu: False


### Dataset Used 
The dataset we are planning to use is the caltech 101 dataset.  It consist of about 40 to 800 images per category. Most categories have about 50 images. Collected in September 2003 by Fei-Fei Li, Marco Andreetto, and Marc'Aurelio Ranzato. The size of each image is roughly 300 x 200 pixels. 

In [3]:
image_paths = list(paths.list_images('/kaggle/input/bike-horse1'))

In [4]:
data = []
labels = []
label_names = []
for image_path in image_paths:
    label = image_path.split(os.path.sep)[-2]
    if label == 'BACKGROUND_Google':
        continue

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    data.append(image)
    label_names.append(label)
    labels.append(label)

In [5]:
import warnings
warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)            

In [6]:
data = np.array(data)
labels = np.array(labels)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (179,) + inhomogeneous part.

Labels are in string format and we need to get it to number format 

In [7]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [8]:
(X, X_Validation , Y, Y_Validation) = train_test_split(data, labels, 
                                                    test_size=0.2,  
                                                    stratify=labels,
                                                    random_state=42)

(x_train, x_test, y_train, y_test) = train_test_split(X, Y, 
                                                    test_size=0.25, 
                                                    random_state=42)

print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nX_Validation examples: {X_Validation.shape}")

AttributeError: 'list' object has no attribute 'shape'

We need to do transformations on the images from this dataset as the model we are intending to use is the AlexNet model trained on ImageNet where the image is of size 224x224. We rescale the image as well as noramise it based on imagenet standards. Other than that since images per class is too low we also try some data augmentation on the train set explictly 

In [9]:
train_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.RandomRotation((-30, 30)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])
val_transform = transforms.Compose(
    [transforms.ToPILImage(),
	 transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

Custom Dataset class which allows us to apply transformations and create a dataset object to pass into dataloader 

In [10]:
# custom dataset
class ImageDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i][:]
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [11]:
train_data = ImageDataset(x_train, y_train, train_transform)
val_data = ImageDataset(X_Validation, Y_Validation, val_transform)
test_data = ImageDataset(x_test, y_test, val_transform)
 
# dataloaders
trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
valloader = DataLoader(val_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16, shuffle=False)

### Pre-trained Model Alexnet 

In [12]:
model = models.alexnet(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 129MB/s]  


In [13]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [14]:
for param in model.parameters():
    param.requires_grad = False

In [15]:
model.classifier[6] = nn.Linear(in_features=4096,out_features=101,bias=True)

In [16]:
for param in model.classifier[6].parameters():
  param.requires_grad = True

In [17]:
model 

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Define your loss function
criterion = nn.CrossEntropyLoss()

In [20]:
epochs = 10 

#### Training and Validation 

In [21]:
def fit(model, dataloader):
    print('Training')
    model.train()
    Train_loss = 0.0
    train_running_correct = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, torch.max(target, 1)[1])
        Train_loss += loss.item()
        _, prediction = torch.max(outputs.data, 1)
        train_running_correct += (prediction == torch.max(target, 1)[1]).sum().item()
        loss.backward()
        optimizer.step()
        
    Train_loss = Train_loss/len(dataloader.dataset)
    Train_acc = 100. * train_running_correct/len(dataloader.dataset)
    
    print(f"Train Loss: {Train_loss:.4f}, Train Acc: {Train_acc:.2f}")
    
    return Train_loss, Train_acc

In [22]:
#validation function
def validate(model, dataloader):
    print('Validating')
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, target = data[0].to(device), data[1].to(device)
            outputs = model(data)
            loss = criterion(outputs, torch.max(target, 1)[1])
            
            val_running_loss += loss.item()
            _, prediction = torch.max(outputs.data, 1)
            val_running_correct += (prediction == torch.max(target, 1)[1]).sum().item()
        
        Validation_loss = val_running_loss/len(dataloader.dataset)
        Validation_acc = 100. * val_running_correct/len(dataloader.dataset)
        print(f'Val Loss: {Validation_loss:.4f}, Val Acc: {Validation_acc:.2f}')
        
        return Validation_loss, Validation_acc

In [23]:
print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")
Train_loss , Train_acc = [], []
Validation_loss , Validation_acc = [], []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    Validation_epoch_loss, Validation_epoch_acc = validate(model, valloader)
    train_epoch_loss, train_epoch_accuracy = fit(model, trainloader)
    Validation_loss.append(Validation_epoch_loss)
    Validation_acc.append(Validation_epoch_acc)
    Train_acc.append(train_epoch_accuracy)
    Train_loss.append(train_epoch_loss)


Training on 107 examples, validating on 36 examples
Epoch 1 of 10
Validating


3it [00:01,  3.00it/s]                       


Val Loss: 0.3890, Val Acc: 0.00
Training


7it [00:02,  2.95it/s]                       


Train Loss: 0.0489, Train Acc: 85.05
Epoch 2 of 10
Validating


...

Train Loss: 0.0000, Train Acc: 100.00
Epoch 10 of 10
Validating


3it [00:01,  2.44it/s]                       


Val Loss: 0.0000, Val Acc: 100.00
Training


7it [00:02,  2.94it/s]                       

Train Loss: 0.0000, Train Acc: 100.00


### Test

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, target = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == torch.max(target, 1)[1]).sum().item()

print('Accuracy of the network on test images: %0.3f %%' % (
    100 * correct / total))

Accuracy of the network on test images: 100.000 %
